In [ ]:
# |default_exp chroma_db

## Install dependencies

## Make an app with Gradio

In [29]:
# |export
import csv
import re
import chromadb
from chromadb import Settings
from dotenv import load_dotenv
import os

import gradio as gr
from fastcore.net import urljson, HTTPError

In [16]:
load_dotenv()
oai_key = os.getenv('GEMINI_API_KEY')

In [8]:
def convert_qa_to_csv(input_file, output_file):
    """
    Convert a text file with Q/A format to a CSV file.

    Args:
        input_file: Path to the input text file
        output_file: Path to the output CSV file
    """
    # Read the content of the file
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the content by 'Q' marker
    qa_blocks = content.split('Q\n')

    qa_blks = [block.strip() for block in qa_blocks][1:]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [[line for line in block.split('\n') ] for block in qa_blks if block.strip()]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [blk for block in qa_blocks if block.strip() for blk in block.strip()]

    # Process each Q&A block
    qa_pairs = []
    for block in qa_blks:
        # Split the block into lines
        lines = block.strip().split('\n')

        if lines:
            # First line is the question
            question = lines[0]
            # The rest are the answer
            answer = '\n'.join(lines[1:])

            # Add the pair to our list
            qa_pairs.append([question, answer])

    # Write to CSV
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # Write header
        writer.writerow(['Question', 'Answer'])
        # Write Q&A pairs
        for pair in qa_pairs:
            writer.writerow(pair)
    print(f"Conversion complete. CSV file saved to {output_file}")

In [9]:

input_files = ["../res/qa_service.txt", "../res/qa_technology.txt"]
output_files = ["../res/qa_service.csv", "../res/qa_technology.csv"]
for in_f, ot_f in zip(input_files, output_files):
    convert_qa_to_csv(in_f, ot_f)
    

Conversion complete. CSV file saved to ../res/qa_service.csv
Conversion complete. CSV file saved to ../res/qa_technology.csv


In [18]:
# |export
client = chromadb.PersistentClient(path="../db")
collections = [client.create_collection(name="siasun_qa_service"),
                client.create_collection(name="siasun_qa_technology")]

In [24]:
i=2
f'q{i}'

'q2'

In [25]:
# |export

for csv_file,collection in zip(output_files,collections):
    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            collection.add(
                documents = row,
                metadatas = [{"source": "question"}, {"source": "answer"}],
                ids = [f"{collection.name}_q{i}", f"{collection.name}_a{i}"]
            )

In [27]:
#|export
results = collections[0].query(
    query_texts=["多久维护一次产品?"],
    n_results=2
)

In [28]:
results

{'ids': [['siasun_qa_service_q2', 'siasun_qa_service_a9']],
 'embeddings': None,
 'documents': [['你们的产品，多久需要维护一次？维护保养内容有哪些？',
   '作业作为一个重要的单元，所以不能进行批量删除与添加，防止误操作造成损失。']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'question'}, {'source': 'answer'}]],
 'distances': [[0.4168012525151972, 0.8514458305258445]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [39]:
results['metadatas'][0] #[0]['source']
results['documents'][0][0]

['你们的产品，多久需要维护一次？维护保养内容有哪些？', '作业作为一个重要的单元，所以不能进行批量删除与添加，防止误操作造成损失。']

In [ ]:
def qa(question:str):
    results = []
    for collection in collections:
        results = collections.query(
            query_texts=["多久维护一次产品?"],
            n_results=4
        )
        docs = []
        for i,metadata in enumerate(results['metadatas'][0]):
            if metadata['source'] == 'question':
                docs.append({'id': results['ids'][0][i],
                                'document': results['documents'][0][i], 
                                'distance':results['distances'][0][i]})
                
    # question = 
    # return answers
                
        


In [ ]:
# |export
iface = gr.Interface(fn=size, inputs=gr.Text(value="tglcourse/CelebA-faces-cropped-128"), outputs="text")
iface.launch(width=500)
demo.launch()

In [ ]:
# this is only necessary in a notebook
demo.close()

## Create a `requirements.txt` file

In [ ]:
%%writefile ../requirements.txt
fastcore

## Convert this notebook into a Gradio app

In [ ]:
# from nbdev.export import nb_export
# nb_export('01_gradio.ipynb', lib_path='.', name='gradio')

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

In [1]:
# |default_exp data_preprocessing

## Install dependencies

In [1]:
# |export
import csv
import re

In [2]:
 l = [22, 13, 45, 50, 98, 69, 43, 44, 1]

In [3]:
 [x+1 if x >= 45 else x+5 for x in l]

[27, 18, 46, 51, 99, 70, 48, 49, 6]

In [5]:
 [x+1 for x in l if x >= 45]

[46, 51, 99, 70]

In [6]:
# |export
def convert_qa_to_csv(input_file, output_file):
    """
    Convert a text file with Q/A format to a CSV file.

    Args:
        input_file: Path to the input text file
        output_file: Path to the output CSV file
    """
    # Read the content of the file
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the content by 'Q' marker
    qa_blocks = content.split('Q\n')

    # Remove empty blocks (like the first one if file starts with 'Q')
    qa_blocks = [blk for block in qa_blocks if block.strip() for blk in block.strip()]

    # Process each Q&A block
    qa_pairs = []
    for block in qa_blocks:
        # Split the block into lines
        lines = block.strip().split('\n')

        if lines:
            # First line is the question
            question = lines[0]
            # The rest are the answer
            answer = '\n'.join(lines[1:])

            # Add the pair to our list
            qa_pairs.append([question, answer])

    # Write to CSV
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # Write header
        writer.writerow(['Question', 'Answer'])
        # Write Q&A pairs
        for pair in qa_pairs:
            writer.writerow(pair)
    print(f"Conversion complete. CSV file saved to {output_file}")


# if __name__ == "__main__":
#     input_file = "res/qa_service.txt"
#     output_file = "res/qa_service.csv"
#     convert_qa_to_csv(input_file, output_file)

In [ ]:
input_files=["res/qa_service.txt", "res/qa_technology.txt"]
outpu

In [5]:
# |export
collection = client.create_collection(name="Students")

In [6]:
# |export
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

In [7]:
# |export
collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student_info"}, {"source": "club_info"}, {"source": "university_info"}],
    ids = ["id1", "id2", "id3"]
)

/home/x/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:45<00:00, 792kiB/s]    


In [9]:
#|export
results = collection.query(
    query_texts=["What is the student name?"],
    n_results=2
)

In [10]:
results

{'ids': [['id1', 'id2']],
 'embeddings': None,
 'documents': [['\nAlexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,\nis a member of the programming and chess clubs who enjoys pizza, swimming, and hiking\nin her free time in hopes of working at a tech company after graduating from the University of Washington.\n',
   "\nThe university chess club provides an outlet for students to come together and enjoy playing\nthe classic strategy game of chess. Members of all skill levels are welcome, from beginners learning\nthe rules to experienced tournament players. The club typically meets a few times per week to play casual games,\nparticipate in tournaments, analyze famous chess matches, and improve members' skills.\n"]],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'student_info'}, {'source': 'club_info'}]],
 'distances': [[1.294698592479661, 1.3954207576145858]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,

In [8]:
# |export
demo.launch()

NameError: name 'demo' is not defined

In [8]:
# this is only necessary in a notebook
demo.close()

Closing server running on port: 7860


## Create a `requirements.txt` file

In [8]:
%%writefile ../requirements.txt
fastcore

Writing ../requirements.txt


## Convert this notebook into a Gradio app

In [9]:
# from nbdev.export import nb_export
# nb_export('01_gradio.ipynb', lib_path='.', name='gradio')

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()